In [1]:
!pip install ../input/vitkerasprerequisites/validators-0.18.2-py3-none-any.whl
!pip install ../input/vitkerasprerequisites/vit_keras-0.1.0-py3-none-any.whl
!pip install ../input/libjpeg-gdcm/pylibjpeg_libjpeg-1.3.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install ../input/libjpeg-gdcm/python_gdcm-3.0.15-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl

Processing /kaggle/input/vitkerasprerequisites/validators-0.18.2-py3-none-any.whl
Processing /kaggle/input/vitkerasprerequisites/vit_keras-0.1.0-py3-none-any.whl
Processing /kaggle/input/libjpeg-gdcm/pylibjpeg_libjpeg-1.3.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/libjpeg-gdcm/python_gdcm-3.0.15-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl


In [2]:
import numpy as np
import pandas as pd
import cv2
import pydicom 
import matplotlib.pyplot as plt
import glob
import os
import random
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.utils import Sequence
from tensorflow import keras
from vit_keras import vit, utils
import tensorflow_addons as tfa
import gc
gc.enable()

In [3]:
cfg = {
    'test_df': '../input/rsna-2022-cervical-spine-fracture-detection/test.csv',
    'batch_size': 32,
    'img_size': 258, #after application of 1st conv layer the output img size = img_size - 2
    'test_imgs': '../input/rsna-2022-cervical-spine-fracture-detection/test_images',
    'model_path': '../input/vit-30k-to-45k-new-loss-album-augs/best_model.h5'
}

def competiton_loss(y_true, y_pred):

    competition_weights = {
        '-' : tf.constant([1, 1, 1, 1, 1, 1, 1, 7], dtype=tf.float32),
        '+' : tf.constant([2, 2, 2, 2, 2, 2, 2, 14], dtype=tf.float32)
    }
    
    loss = tf.keras.losses.BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)(tf.expand_dims(y_true, -1),tf.expand_dims(y_pred,-1))
    weights  = y_true*competition_weights['+'] + (1-y_true)*competition_weights['-'] 
    
    loss = tf.reduce_mean(tf.reduce_sum(loss * weights, axis=1)) / tf.reduce_sum(weights)
    return loss

model = tf.keras.models.load_model(cfg['model_path'], custom_objects={'competiton_loss': competiton_loss})

2022-10-08 13:16:29.195191: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-08 13:16:29.297990: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-08 13:16:29.298810: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-08 13:16:29.300010: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [4]:
submission = {'row_id': [], 'fractured':[]}

def callableFunction():
    return 0.1

subs = ['_patient_overall', '_C1', '_C2', '_C3', '_C4', '_C5', '_C6', '_C7']

files = os.listdir(cfg['test_imgs'])
for file in tqdm(files):
    samples = 0; sums = np.zeros(shape=(1, 8))
    _dir = os.path.join(cfg['test_imgs'], file)
    instances = os.listdir(_dir)
    random.shuffle(instances, callableFunction)
    for idx, instance in enumerate(instances):
        data = pydicom.dcmread(os.path.join(_dir, instance))
        img = pydicom.pixel_data_handlers.util.apply_voi_lut(data.pixel_array, data)
        img = cv2.resize(img, (cfg['img_size'], cfg['img_size'])).reshape((1, cfg['img_size'], cfg['img_size'], 1))
        sums += model.predict(img); samples+=1
        if samples==50:
            break
    sums = sums / samples
    sums = sums.tolist()[0]
    for i in subs:
        submission['row_id'].append(file + i)
    submission['fractured'].append(sums[-1])
    for i in range(7):
        submission['fractured'].append(sums[i])

  0%|          | 0/3 [00:00<?, ?it/s]

2022-10-08 13:16:45.482434: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-10-08 13:16:48.782360: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


In [5]:
submission = pd.DataFrame(submission)
submission.to_csv('submission.csv', index=False)
df = pd.read_csv('./submission.csv')
df.head()

,row_id,fractured
0,1.2.826.0.1.3680043.22327_patient_overall,0.729333
1,1.2.826.0.1.3680043.22327_C1,0.150066
2,1.2.826.0.1.3680043.22327_C2,0.292803
3,1.2.826.0.1.3680043.22327_C3,0.122322
4,1.2.826.0.1.3680043.22327_C4,0.164326
